<h1>NN Model</h1>

In [1]:
from pycocotools.coco import COCO
import matplotlib
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as functions
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import transforms
import re

In [2]:
DATADIR = "cocodoom/"
USED_RUNS = ["run1", "run2", "run3"]

dataSplit, TRAIN_RUN = "run-full-train", "run1"

annFile = '{}{}.json'.format(DATADIR,dataSplit)

In [3]:
coco_train = COCO(annFile)

loading annotations into memory...
Done (t=24.13s)
creating index...
index created!


In [4]:
dataSplit, TEST_RUN = "run-full-test", "run3"

annFile = '{}{}.json'.format(DATADIR,dataSplit)

In [5]:
coco_test = COCO(annFile)

loading annotations into memory...
Done (t=32.98s)
creating index...
index created!


In [45]:
player_positions = {"run1":{}, "run2":{}, "run3":{}}

for run in USED_RUNS:
    with open(DATADIR+run+"/log.txt", 'r') as log_file:
        for line in log_file:
            if "player" in line:
                line = line.strip()
                tic, stats = line.split("player:")
                x, y, z, angle = stats.split(",")
    
                # Store position in the dictionary
                player_positions[run][int(tic)] = (float(x), float(y), float(z), float(angle))

In [46]:
class DoomMotionDataset(Dataset):
    def __init__(self, coco, run, transform=None):
        self.coco = coco
        self.run = run
        self.img_ids = self.coco.getImgIds()
        self.transform = transform

    def __len__(self):
        return len(self.img_ids)

    def __getitem__(self, idx):
        # Load the RGB image
        rgb_filename = self.coco.loadImgs(self.img_ids[idx])[0]['file_name']
        tic = int(rgb_filename.replace(".png", "").split("/")[-1])
        next_tic = tic+1
        previous_tic = tic-1

        player_position = player_positions[self.run][tic]
        
        if next_tic not in player_positions[self.run]:
            next_player_position = player_position
        else:
            next_player_position = player_positions[self.run][next_tic]
        dx = next_player_position[0] - player_position[0]
        dy = next_player_position[1] - player_position[1]
        dz = next_player_position[2] - player_position[2]
        np.pi - abs(abs(next_player_position[3] - player_position[3]) - np.pi)
        next_motion_vector = (dx, dy, dz, dangle)

        if previous_tic not in player_positions[self.run]:
            prev_player_position = player_position
        else :
            prev_player_position = player_positions[self.run][previous_tic]
        dx = player_position[0] - prev_player_position[0]
        dy = player_position[1] - prev_player_position[1]
        dz = player_position[2] - prev_player_position[2]
        np.pi - abs(abs(player_position[3] - prev_player_position[3]) - np.pi)
        prev_motion_vector = (dx, dy, dz, dangle)


        # if dx > 1000:
        #     print(f"idx: {idx}")
        #     print(f"rgb_filename: {rgb_filename}")
        #     print(f"tic: {tic}")
        #     print(f"next_tic: {next_tic}")
        #     print(f"previous_tic: {previous_tic}")
        #     print(f"Sus {idx}")
        #     print(f"prev_player_position: {prev_player_position}")
        #     print(f"player_position: {player_position}")
        #     print(f"next_player_position: {next_player_position}")
        #     print(f"prev_motion_vector: {prev_motion_vector}")
        #     print(f"next_motion_vector: {next_motion_vector}")
            
        prev_motion_vector = torch.tensor(prev_motion_vector, dtype=torch.float32)
        next_motion_vector = torch.tensor(next_motion_vector, dtype=torch.float32)
        
        return prev_motion_vector, next_motion_vector


In [47]:
class NeuralNetwork(nn.Module):
  def __init__(self, activation_function=functions.relu, device=torch.device("cpu")):
    super(NeuralNetwork, self).__init__()

    self.activation_function = activation_function
    self.fc1 = nn.Linear(4, 64).to(device)
    self.bn1 = nn.BatchNorm1d(num_features=64).to(device)

    self.fc2 = nn.Linear(64, 128).to(device)
    self.bn2 = nn.BatchNorm1d(num_features=128).to(device)

    self.fc3 = nn.Linear(128, 64).to(device)
    self.bn3 = nn.BatchNorm1d(num_features=64).to(device)

    self.out = nn.Linear(64, 4).to(device)

  def forward(self, x):
    x = self.activation_function(self.bn1(self.fc1(x)))
    x = self.activation_function(self.bn2(self.fc2(x)))
    x = self.activation_function(self.bn3(self.fc3(x)))
    x = self.out(x)
    return x

In [48]:
batch_size = 256
learning_rate = 1e-3
num_epochs = 300
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("cuda" if torch.cuda.is_available() else "cpu")
model = NeuralNetwork(device=device).to(device)

train_dataset = DoomMotionDataset(coco_train, TRAIN_RUN)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

test_dataset = DoomMotionDataset(coco_test, TEST_RUN)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")
    for batch_idx, (inputs, targets) in enumerate(progress_bar):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
    
        loss = criterion(outputs, targets)
        loss.backward()

        optimizer.step()

        running_loss += loss.item()

        progress_bar.set_postfix({
            "batch_loss": loss.item(),
            "batch_index": batch_idx + 1,
            "batch_size": inputs.size(0)
        })

    # Average loss per epoch
    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")

    # model.eval()  # Set the model to evaluation mode
    # running_loss = 0.0
    
    
    # progress_bar = tqdm(test_loader, desc="Testing", unit="batch")
    
    # with torch.no_grad():  # Disable gradient calculations for evaluation
    #     for batch_idx, (inputs, targets) in enumerate(progress_bar):
    #         inputs, targets = inputs.to(device), targets.to(device)
            
    #         outputs = model(inputs)
    #         loss = criterion(outputs, targets)
            
    #         running_loss += loss.item()
            
    #         progress_bar.set_postfix({
    #             "batch_loss": loss.item(),
    #             "batch_index": batch_idx + 1,
    #             "batch_size": inputs.size(0)
    #         })
    
    # # Average loss over all batches
    # test_loss = running_loss / len(test_loader)
    # print(f"Test Loss: {test_loss:.4f}")

# Save the trained model
torch.save(model.state_dict(), "movement_nn.pth")

cuda



poch 1/300: 100%|██████████████| 991/991 [00:17<00:00, 58.19batch/s, batch_loss=15.9, batch_index=991, batch_size=220]

Epoch 1, Loss: 473.8735



poch 2/300: 100%|██████████████| 991/991 [00:16<00:00, 60.68batch/s, batch_loss=15.8, batch_index=991, batch_size=220]

Epoch 2, Loss: 471.6425


...

Epoch 298, Loss: 325.6620



poch 299/300: 100%|████████████| 991/991 [00:16<00:00, 60.56batch/s, batch_loss=24.7, batch_index=991, batch_size=220]

Epoch 299, Loss: 319.2989



poch 300/300: 100%|████████████| 991/991 [00:17<00:00, 58.20batch/s, batch_loss=21.5, batch_index=991, batch_size=220]

Epoch 300, Loss: 320.5357


In [49]:
device = (torch.device("cuda" if torch.cuda.is_available() else "cpu"))
criterion = torch.nn.MSELoss()
model = NeuralNetwork(device="cuda").to(device)
model.load_state_dict(torch.load("doom_motion_model.pth", weights_only=True))

test_dataset = DoomMotionDataset(coco_test, TEST_RUN)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

model.eval()  # Set the model to evaluation mode
running_loss = 0.0


progress_bar = tqdm(test_loader, desc="Testing", unit="batch")
losses = []

with torch.no_grad():  # Disable gradient calculations for evaluation
    for batch_idx, (inputs, targets) in enumerate(progress_bar):
        inputs, targets = inputs.to(device), targets.to(device)
        
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        losses.append([batch_idx, outputs, loss])
        
        running_loss += loss.item()
        
        progress_bar.set_postfix({
            "batch_loss": loss.item(),
            "batch_index": batch_idx + 1,
            "batch_size": inputs.size(0)
        })

# Average loss over all batches
test_loss = running_loss / len(test_loader)
print(f"Test Loss: {test_loss:.4f}")

print(f"Biggest Loss: {losses.sort(key=lambda x:x[2])[0][2]}")


esting: 100%|█████████| 118138/118138 [09:41<00:00, 203.11batch/s, batch_loss=0.154, batch_index=118138, batch_size=1]

Test Loss: 1960.0671


TypeError: 'NoneType' object is not subscriptable

In [42]:
print(losses[-1])
rgb_filename = coco_test.loadImgs(coco_test.getImgIds()[losses[-1][0]])[0]['file_name']
tic = int(rgb_filename.replace(".png", "").split("/")[-1])
prev_player_position = player_positions[TEST_RUN][tic - 1]
player_position = player_positions[TEST_RUN][tic]
next_player_position = player_positions[TEST_RUN][tic + 1]
dx = next_player_position[0] - player_position[0]
dy = next_player_position[1] - player_position[1]
dz = next_player_position[2] - player_position[2]
dangle = np.pi - abs(abs(next_player_position[3] - player_position[3]) - np.pi)
print(f"Actual change {dx, dy, dz, dangle}")
print(next_player_position)
print(player_position)
print(prev_player_position)

[98000, tensor([[1.3958e+03, 6.7715e+03, 4.3515e+01, 4.8347e+00]], device='cuda:0'), tensor(11950945., device='cuda:0')]
Actual change (0.0, 0.0, 0.0, 0.0)
(-1760.0, 1504.0, 65.0, 4.71239)
(-1760.0, 1504.0, 65.0, 4.71239)
(-2144.36, -3444.42, 110.722, 5.12963)


In [32]:
print(losses[-2])

[25935, tensor([[-4715.9492, -3745.4646,   330.2908,     5.8413]], device='cuda:0'), tensor(9094451., device='cuda:0')]


In [33]:
print(losses[0])

[20619, tensor([[ 0.5578,  0.0200, -0.1332,  0.4816]], device='cuda:0'), tensor(0.0006, device='cuda:0')]


In [34]:
print(len([i for i in losses if i[2] > 100000]))

177


In [10]:
print(player_positions[3132])

(799.822, 1304.14, 89.0002, 4.36878)


In [37]:
print(coco_test.loadImgs(coco_test.getImgIds()[98000])[0]['file_name'])

run3/map26/rgb/099653.png


In [ ]:
print